# Text classification using CNNs

`CNNs(Convolutional Neural Networs)` 구조를 이용하여 문장 분류를 해보도록 하겠습니다. 데이터는 `Pang & Lee (2004)`에서 사용한 subjectivity dataset을 이용하였습니다. (http://www.cs.cornell.edu/people/pabo/movie-review-data/)

## Data

**subjectivity** dataset은 5000개의 subjectivity 문장과 5000개의 objectivity 문장으로 구성되어 있습니다. subjectivity 문장들은 [Rotten Tomatoes](http://www.rottentomatoes.com)에 남겨진 영화 리뷰에서 가지고 온 문장들이며, objectivity 문장들은 [IMDB](http://www.imdb.com)에서 요약된 영화 줄거리에서 가지고 온 문장입니다. 실제 데이터를 수집한 저자들은 대부분의 경우 해당 레이블이 일치하였으나, 영화 줄거리 중 몇 개의 데이터가 실제로는 subjectivity일 수도 있다고 설명합니다. 

lab4-0 notebook을 이용하여 두 개의 텍스트 파일을 불러와서 **`data/subjectivity/massaged_subjectivity.txt`** 파일로 저장하였습니다. 해당 파일은 **label'\t'sentence**의 형태로 두 개의 텍스트 파일을 합쳐놓았습니다. 


## Convolutional Neural Networs for Sentence Classification

`CNN` 모델을 이용하여 텍스트를 분류하는 것은 `Yoon (2014)`(https://arxiv.org/abs/1408.5882)에 잘 설명이 되어있습니다. 아래 그림을 통해 간략하게 구조를 살펴보겠습니다. 


![alt text](resources/cnn_structure.png "CNN structure in Yoon (2014)")

CNN은 인접한 계층의 연결성을 활용하여서 이미지의 공간 구조를 파악하였습니다. 문장 분류에서도 마찬가지입니다. 인접 계층의 연결성을 활용하여서 단어간의 공간 구조를 이용하여 문맥 정보를 유지하고, 그를 바탕으로 문장 분류를 하는 것입니다. 이번에는 `Yoon (2014)`의 구조와 유사하지만 훨씬 간단한 구조를 만들어서 살펴보도록 하겠습니다. 

기존의 **language model**에서는 `n-gram` 모델을 활용하여 단어의 문맥 구조를 파악하였습니다. 지금 사용할 CNN에서도 이와 유사한 과정이 있습니다. 한 번에 전체를 보는 것이 아니라 적절한 `필터 크기/커널 크기(filter/kernel size)`를 부여하여서 해당 범위 안의 단어를 살펴보는 `convolution filter`를 학습하게 되고, `max-pooling`을 통해서 문장에서 가장 중요한 특징(feature)을 나타내는 벡터를 얻게 됩니다. 

이미지의 경우 크기가 동일한 데이터를 많이 구할 수 있지만, 텍스트 데이터의 경우 문장의 길이를 특정짓기가 매우 어렵습니다. 그렇기 때문에 대부분의 경우 가장 긴 문장의 길이만큼의 배열로 설정한 이후, 문장의 길이가 짧을 경우 남은 공간을 `zero padding`을 하거나 `unknown word`로 처리하게 됩니다. 

이전에 `word embedding`에서와 마찬가지로 단어들은 각각의 index로 변환되어야 합니다. index로 변환된 단어들은 해당 단어의 벡터값으로 변환된 후, 입력값으로 사용됩니다. 입력값은 `Convolution layer`로 연결된 이후, `filter/kernel size`에 해당하는 단어들의 값을 미리 설정한 `conv_dim`만큼의 다른 방법으로 연산합니다. 각각의 `conv_dim`의 값들은 `max-pooling`의 과정을 통해 각각 하나의 값을 산출하고, `Fully connected layer`의 구조로 `구분할 클래스의 갯수`만큼의 노드로 출력하게 됩니다. `softmax` 활성화를 하게 되면 출력값은 확률로 변화되는데, 하나는 **subjectivity**, 다른 하나는 **objectivity**에 해당하는 값입니다. 

## Import modules

필요한 모듈을 불러오고, 재실행을 대비하여 랜덤 시드를 설정하겠습니다. 

In [4]:
from gensim.models import KeyedVectors

from keras.layers.core import Dense, Dropout, SpatialDropout1D
from keras.layers.convolutional import Conv1D
from keras.layers.embeddings import Embedding
from keras.layers.pooling import GlobalMaxPooling1D

from keras.models import Sequential

from keras.preprocessing.sequence import pad_sequences

from keras.utils import np_utils

from sklearn.model_selection import train_test_split

import collections
import matplotlib.pyplot as plt
import nltk
import numpy as np
import codecs

%matplotlib inline

np.random.seed(42)

Using TensorFlow backend.


nltk를 불러온 이후, word_tokenizer를 실행하기 위해서는 `punkt` 패키지가 필요합니다. 아래 셀을 실행하여서 다운받겠습니다. 

In [15]:
%%bash
python3 -m nltk.downloader -d /usr/local/share/nltk_data -u https://pastebin.com/raw/D3TBY4Mj punkt

Process is terminated.


## Hyperparameters

필요한 파라미터들을 설정하도록 하겠습니다. 

In [2]:
data_file = "data/subjectivity/massaged_subjectivity.txt"
model = "data/GoogleNews-vectors-negative300.bin"
embed_dim = 300  # word2vec 모델에서 설정된 embedding 차원의 값입니다.
hidden_dim = 256 # Filter를 몇 개의 방법으로 연산할지 정합니다
filter_size = 3  # Filter/kernel size 값입니다
batch_size = 64  # 한 번에 몇 개의 데이터를 처리할 지 정합니다
num_epoch = 10   # 전체 데이터를 몇 번 반복해서 학습할지 정합니다.

## Other parameters

위에서 설정한 파라미터 이외에도 다음의 정보가 필요합니다. 해당 값들은 임의로 설정할 수 없고, 데이터를 분석하여 산출하여야 합니다. 
 1. 사용된 단어의 수
 2. 단어-인덱스 관계를 정의한 `<dict>`
 3. 인덱스-단어 관계를 정의한 `<dict>`

In [6]:
## Count the number of words
counter = collections.Counter()
f = open(data_file, "rb")
seq_len = 0
for line in f:
    line = line.decode("ascii", "ignore")
    _, sent = line.strip().split("\t")
    words = [x.lower() for x in nltk.word_tokenize(sent)]
    if len(words) > seq_len:
        seq_len = len(words)
    for word in words:
        counter[word] += 1
f.close()

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  Searched in:
    - '/root/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - ''
**********************************************************************
